In [1]:
import pandas as pd
import datetime
import requests
import tabula
import io
import os
import sys

#URLlib
import urllib
import urllib.request

#PDFminer
import pdfminer
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from io import StringIO
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal

In [2]:
#Palabras a buscar 
word_count = "Word Count"
letter_count = "Letter Count"
word_1 = "Crecimiento"
word_2 = "Riesgo"
word_3 = "inflacion"
#Misma formato para mas palabras


In [3]:
word_seek = pd.DataFrame( index = [word_count,
                                  letter_count,
                                  word_1,
                                  word_2,
                                  word_3])

In [4]:
#df2 = tabula.read_pdf("https://www.banxico.org.mx/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria/%7B1170CD25-C24D-E99C-F3D3-6B86D1822E65%7D.pdf")

In [5]:
#Import Statement
#url = "https://www.banxico.org.mx/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria/%7B1170CD25-C24D-E99C-F3D3-6B86D1822E65%7D.pdf"
#pdf_file = requests.get(url, stream = True)
#print(pdf_file)

In [6]:
path = "statement.pdf"

In [7]:
class MyParser(object):
    def __init__(self, pdf):
        parser = PDFParser(cStringIO.StringIO(pdf_file.read()))
        document = PDFDocument(parser)
        rsrcmngr = PDFResourceManager()
        retstr = StringIO()
        laprams = LAParams()
        codec = 'ut - 8'
        

In [8]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    #codec = 'utf - 8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [9]:
convert_pdf_to_txt(path)

' \n\n14 de noviembre de 2019 \n\nAnuncio de Política Monetaria  \n\nComunicado de Prensa \n\nLa Junta de Gobierno del Banco de México ha decidido disminuir en 25 puntos base el objetivo \npara la Tasa de Interés Interbancaria a un día a un nivel de 7.50%. \n \nDurante el tercer trimestre del año continuó la desaceleración de la economía mundial y sus perspectivas \nde crecimiento han seguido revisándose a la baja. Ello, en un entorno en el que las inflaciones general \ny subyacente en las principales economías avanzadas permanecen en niveles inferiores a las metas de \nsus respectivos bancos centrales. En este contexto, algunas de estas instituciones han tomado acciones \nde relajamiento monetario en los últimos meses. En su reunión de octubre la Reserva Federal redujo en \n25 puntos base el rango objetivo para la tasa de fondos federales y en su comunicación destacó que el \nrango  alcanzado  es  apropiado  considerando  los  riesgos  que  enfrenta.  Por  su  parte,  los  mercados \n